#### iterator與dataset關係:https://www.twblogs.net/a/5b8e85d62b7177188345d355

## Dataset和Iterator關係
#### 如果Dataset是一個水池的話,數據就如同水池中的水,Iterator可以當作一個水管,在tensorflow裡,正式通過Iteraotr這根水管,源源不斷的從Dataset中取出數據,為了應付多變的環境,水管也需要變化,Iterator也有許多種類

#### 構建了表示輸入數據的Dataset後，下一步就是創建Iterator來訪問該數據集中的元素。tf.data API目前支持下列迭代器，其複雜程度逐漸上升：

#### （1）單次迭代器(一次性水管)
#### （2）可初始化迭代器(訂製水管)
#### （3）可重新初始化迭代器(能接不同水池的水管)
#### （4）可feeding迭代器(水管的轉換器)

# （1）單次迭代器(一次性水管)
#### 單次迭代器是最簡單的迭代器形式，僅支持對數據集進行一次迭代，不需要顯式初始化。單次迭代器可以處理現有的基於隊列的輸入管道支持的幾乎所有情況，但不支持參數化。以Dataset.range()為例：

In [1]:
import tensorflow as tf

dataset = tf.data.Dataset.range(5)
iterator = dataset.make_one_shot_iterator() ##創建單次迭代器

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(iterator.get_next()))
        except tf.errors.OutOfRangeError:
            break

0
1
2
3
4


'上面的代碼非常簡單，首先創建了一個包含 0 到 4 的數據集。然後，創建了一個單次迭代器。\n\n通過循環調用 get_next() 方法就可以將數據取出。\n\n需要注意的是，通常用 try-catch 配合使用，當 Dataset 中的數據被讀取完畢的時候，程序會拋出異常，獲取這個異常就可以從容結束本次數據的迭代。\n\n然後， iterator 就完成了它的歷史使命。單次的迭代器，不支持動態的數據集，它比較單純，它不支持參數化。\n\n什麼是參數化呢？你可以理解爲單次的 Iterator 認死理，它需要 Dataset 在程序運行之前就確認自己的大小，但我們都知道 Tensorflow 中有一種 feeding 機制，它允許我們在程序運行時再真正決定我們需要的數據，很遺憾，單次的 Iterator 不能滿足這要的要求。'

#### 上面的代碼非常簡單，首先創建了一個包含 0 到 4 的數據集。然後，創建了一個單次迭代器。
#### 通過循環調用 get_next() 方法就可以將數據取出。
#### 需要注意的是，通常用 try-catch 配合使用，當 Dataset 中的數據被讀取完畢的時候，程序會拋出異常，獲取這個異常就可以從容結束本次數據的迭代。
#### 然後， iterator 就完成了它的歷史使命。單次的迭代器，不支持動態的數據集，它比較單純，它不支持參數化。
##什麼是參數化呢？你可以理解爲單次的 Iterator 認死理，它需要 Dataset 在程序運行之前就確認自己的大小，但我們都知道 Tensorflow 中有一種 feeding 機制，它允許我們在程序運行時再真正決定我們需要的數據，很遺憾，單次的 Iterator 不能滿足這要的要求。

## 參數化,可以看見會報錯
##單次 Iterator 無法滿足參數化的要求，但有其他類型的 Iterator 可以完成這個目標。

In [2]:
import tensorflow as tf

max_value = tf.placeholder(tf.int64,shape=[])
dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_one_shot_iterator()

with tf.Session() as sess:

    while True:
            try:
                print(sess.run(iterator.get_next(),feed_dict={max_value:5}))
            except tf.errors.OutOfRangeError:
                break

ValueError: Cannot capture a placeholder (name:Placeholder, type:Placeholder) by value.


# (2）可初始化迭代器
### 單次 Iterator 無法滿足參數化的要求，但有其他類型的 Iterator 可以完成這個目標。
### 需先透過iterator.initializer初始化iterator，才能使用可初始化迭代器
#### 雖然有些不便，但它允許您使用一個或多個tf.placeholder()張量（可在初始化迭代器時饋送）參數化數據集的定義。繼續以Dataset.range()為例：
#### 上面的程式碼可以改寫為

In [4]:
import tensorflow as tf

max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_initializable_iterator() #創建可初始化迭代器

with tf.Session() as sess:
    #initialize an iterator over a dataset with 10 element
    sess.run(iterator.initializer, feed_dict={max_value: 5})
    while True:
        try:
            print(sess.run(iterator.get_next()))
        except tf.errors.OutOfRangeError:
            break
    #initialize the same iterator over a dataset with 100 element
    sess.run(iterator.initializer, feed_dict={max_value: 6})
    while True:
        try:
            print(sess.run(iterator.get_next()))
        except tf.errors.OutOfRangeError:
            break

0
1
2
3
4
0
1
2
3
4
5


#### output: 01234,012345 

#### 跟單次 Iterator 的代碼只有 2 處不同。

#### 1、創建的方式不同，iterator.make_initialnizer()。

#### 2、每次重新初始化的時候，都要調用sess.run(iterator.initializer)

#### 你可以這樣理解，Dataset 這個水池連續裝了 2 次水，每次水量不一樣，但可初始化的 Iterator 很好地處理了這件事情，但需要注意的是，這個時候 Iterator 還是面對同一個 Dataset。

# reinitializable iterator(能夠接不同水池的水管)
#### 有時候，需要一個 Iterator 從不同的 Dataset 對象中讀取數值。Tensorflow 針對這種情況，提供了一個可以重新初始化的 Iterator，它的用法相對而言，比較複雜，但好在不是很難理解。
### 可以通過不同的Dataset進行initialize
#### ex.有一個input,並對其進行隨機擾動泛化數據,並有一個驗證input,評估未修改數據的預測
#### 上述的例子中常會使用不同的Dataset,這些Dataset有相同結構

In [6]:
import tensorflow as tf
import numpy as np
training_dataset = tf.data.Dataset.range(10).map(
    lambda x: x+tf.random.uniform([],-10,10,tf.int64))
validation_dataset = tf.data.Dataset.range(5)

""" 我們可以使用training_dataset或是
validation_dataset的output.types與output.value來定義
reinitializable iterator的結構,兩個是可以相互兼容"""

iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                           training_dataset.output_shapes)

training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

next_element = iterator.get_next()

with tf.Session() as sess:
    for _ in range(3): #Run 3 epochs
        #基於training dataset初始化iterator
        sess.run(training_init_op)
        for _ in range(3):
            print(sess.run(next_element))
        print('===========')
        #基於validation dataset初始化iterator
        sess.run(validation_init_op)
        for _ in range(2):
            print(sess.run(next_element))
        print("===========")    

3
-8
6
0
1
5
8
-4
0
1
-8
6
2
0
1


### 核心代碼
##### iterator = tf.data.Iterator.from_structure(training_data.output_types,training_data.output_shapes)

##### train_op = iterator.make_initializer(training_data)
##### validation_op = iterator.make_initializer(validation_data)
#### Iterator 可以接多個水池裏面的水，但是要求這水池裏面的水是同樣的品質。也就是，多個 Dataset 中它們的元素數據類型和形狀應該是一致的。通過 from_structure() 統一規格，後面的 2 句代碼可以看成是 2 個水龍頭，它們決定了放哪個水池當中的水。


#### 不知道大家注意到一點沒有？每次 Iterator 切換時，數據都從頭開始打印了。如果，不想這種情況發生，就需要接下來介紹的另外一種 Iterator。

# feedable iterator(水管的轉換器)
#### Tensorflow 最美妙的一個地方就是 feeding 機制，它決定了很多東西可以在程序運行時，動態填充，這其中也包括了 Iterator。
#### 不同的 Dataset 用不同的 Iterator，然後利用 feeding 機制，動態決定，聽起來就很棒，不是嗎？

#### 無論是在機器學習還是深度學習當中，訓練集、驗證集、測試集是大家繞不開的話題，但偏偏它們要分離開來，偏偏它們的數據類型又一致，所以，經常我們要寫同樣的重複的代碼。

#### 複用，是軟件開發中一個重要的思想。

#### 可饋送的 Iterator 一定程度上可以解決重複的代碼，同時又將訓練集和驗證集的操作清晰得分離開來。

In [7]:
import tensorflow as tf
import numpy as np

training_dataset = tf.data.Dataset.range(100).map(
    lambda x: x+tf.random.uniform([], -10, 10, tf.int64))
testing_dataset = tf.data.Dataset.range(50)
"""feedable iterator是藉由handle placeholder與其structure所定義
   由於training與testing 兩個dataset有相同sturcture,可使用其output性質初始iterator"""
handle = tf.placeholder(tf.string, shape = [])
iterator = tf.data.Iterator.from_string_handle(
    handle,training_dataset.output_types,training_dataset.output_shapes
)
next_element = iterator.get_next()
#可使用不同類型的iterator來填充 feedable_iterator
training_iterator =  training_dataset.make_one_shot_iterator()
testing_iterator = testing_dataset.make_initializable_iterator()

with tf.Session() as sess:
    #iterator.string_handle()可返回一個tensor來賦值或填充handle這個placeholder
    training_handle = sess.run(training_iterator.string_handle())
    testing_handle = sess.run(testing_iterator.string_handle())
    for _ in range(3):
    
        for _ in range(2):
            print(sess.run(next_element, feed_dict={handle: training_handle}))
        print("==========")
        
        sess.run(testing_iterator.initializer)
        
        for _ in range(5):
            print(sess.run(next_element,feed_dict={handle: testing_handle}))
        
        print("==========")

1
-6
0
1
2
3
4
7
-2
0
1
2
3
4
2
2
0
1
2
3
4


### 看起來跟前面以小節的代碼沒有多大區別。核心代碼如下：
####  handle = tf.placeholder(tf.string,shape=[])
#### iterator = tf.data.Iterator.from_string_handle(
#### handle,train_data.output_types,train_data.output_shapes)
#### train_iterator_handle = sess.run(train_op.string_handle())
#### val_iterator_handle = sess.run(validation_op.string_handle())
#### 它是通過一個 string 類型的 handle 實現的。需要注意的一點是，string_handle() 方法返回的是一個 Tensor，只有運行一個 Tensor 纔會返回 string 類型的 handle。不然，程序會報錯。

####如果用圖表的形式加深理解的話，那就是可饋送 Iterator 的方式，可以自主決定用哪個 Iterator，就好比不同的水池有不同的水管，不需要用同一根水管接到不同的水池當中去了。

#### 可饋送的 Iterator 和可重新初始化的 Iterator 非常相似，但是，可饋送的 Iterator 在不同的 Iterator 切換的時候，可以做到不從頭開始。

# 總結

#### 1、 單次 Iterator ，它最簡單，但無法重用，無法處理數據集參數化的要求。 
#### 2、 可以初始化的 Iterator ，它可以滿足 Dataset 重複加載數據，滿足了參數化要求。 
#### 3、可重新初始化的 Iterator，它可以對接不同的 Dataset，也就是可以從不同的 Dataset 中讀取數據。 
#### 4、可饋送的 Iterator，它可以通過 feeding 的方式，讓程序在運行時候選擇正確的 Iterator,它和可重新初始化的 Iterator 不同的地方就是它的數據在不同的 Iterator 切換時，可以做到不重頭開始讀取數據。

#### 終上所述，在真實的神經網絡訓練過程當中，可饋送的 Iterator 是最值得推薦的方式。